In [ ]:
# %% [markdown]
# # Setup

# %%
# If needed (uncomment in Colab/clean env):
# !pip install -q "openai>=1.101.0" pandas tqdm nest_asyncio

# %%
import os
import json
import asyncio
import pandas as pd
from tqdm import tqdm

# Async OpenAI client (official)
from openai import AsyncOpenAI

# Jupyter event loop helper so we can call async code from normal cells
import nest_asyncio
nest_asyncio.apply()

# IMPORTANT: set your key as an environment variable rather than hardcoding:
# os.environ["OPENAI_API_KEY"] = "sk-..."   # <-- set externally (e.g., in your shell, .env, or here if you must)
aclient = AsyncOpenAI(api_key="...")


In [ ]:
# %% [markdown]
# # Load and prepare data (unchanged)

# %%
import pandas as pd
promises = pd.read_csv("../data/sxp1500_presentations_ceo_aggregated_promises_expanded_cleaned_transcriptlevel_horizon_specificity.csv")# create promise_id column, it is gvkey_transcriptid_2digitnumber (01, 02, 03, ...)
promises['promise_id'] = promises.groupby(['gvkey', 'transcriptid']).cumcount() + 1
promises['promise_id'] = promises['gvkey'].astype(str) + '_' + promises['transcriptid'].astype(str) + '_' + promises['promise_id'].apply(lambda x: f'{x:02d}')
promises_select = promises[['gvkey', 'mostimportantdateutc', 'companyname', 'exec_fullname', 'execid', 'promise_id','1-promise-verbatim' ,'2-promise-explain' ,'3-promise-horizon-v2', 'specificity_score']].sort_values(by=['gvkey', 'mostimportantdateutc',])

labels = pd.read_csv("promises_with_keywords_v5_labels.csv")
def revert_promise_id(promise_id):
    parts = promise_id.split('_')
    fixed_parts = []
    for part in parts:
        if part.endswith('.0'):
            part = str(int(float(part)))
        fixed_parts.append(part)
    return '_'.join(fixed_parts)

labels['promise_id'] = labels['promise_id'].apply(revert_promise_id)

labels = labels[['promise_id', 'primary_keyword']]

# merge promises and labels on promise_id
promises_select = pd.merge(labels, promises_select, on=['promise_id'], how='left')

# %%
promises_select.head(5)


/var/folders/qr/1pq07r555gxgt22gqksl2tlw0000gn/T/ipykernel_80292/2245440812.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  promises = pd.read_csv("../data/LIWC-22 Results - sxp1500_presentations_ceo_aggr___ - LIWC Analysis_v11_horizon_v2_specificity.csv")


,promise_id,primary_keyword,gvkey,mostimportantdateutc,companyname,exec_fullname,execid,1-promise-verbatim,2-promise-explain,3-promise-horizon-v2,specificity_score
0,10353_45212_05,Shareholder and Stakeholder Trust and Engagement,10353,2010-01-06,"Team, Inc.",Phillip J. Hawk,49854,We have always been and continue to be fully c...,The CEO asserts a continuous commitment to eth...,unclear,1.0
1,11600_45248_01,Risk Management,11600,2010-01-06,"Worthington Industries, Inc.",John P. McConnell,3497,But I assure you we will remain vigilant in se...,The CEO is committing that the company will ma...,unclear,1.0
2,25338_45259_01,Experience Enhancement,25338,2010-01-06,Bed Bath & Beyond Inc.,Steven H. Temares,13517,We always look for ways to enhance our custome...,The CEO is affirming a long‐term commitment to...,unclear,1.0
3,25338_45259_02,Investment,25338,2010-01-06,Bed Bath & Beyond Inc.,Steven H. Temares,13517,While we continue to review and prioritize our...,The CEO is committing to allocate necessary ca...,unclear,1.0
4,146017_45297_01,Pricing and Rate Management,146017,2010-01-06,"Acuity Brands, Inc.",Vernon J. Nagel,11057,"However, as I have said before, we will defend...",The CEO is committing the company to actively ...,unclear,1.0


In [9]:
# %% [markdown]
# # Use ChatGPT to Search (now async & concurrent; minimal code changes otherwise)

# %%
STATUS_MAP = {
    "ON_TIME": "delivered on time",
    "DELAYED": "delayed delivery",
    "NOT_DELIVERED": "not delivered",
    "UNMEASURABLE": "it is tough to define what a promise delivery is",
    "NO_EVIDENCE": "it is easy to define promise delivery but I couldn't find relevant data on it",
}

JSON_SCHEMA = {
    "name": "promise_audit",
    "strict": True,
    "schema": {
        "type": "object",
        "additionalProperties": False,
        "properties": {
            "status_code": {"type": "string", "enum": list(STATUS_MAP.keys())},
            "delivery": {"type": "string", "enum": list(STATUS_MAP.values())},
            "explanation": {"type": "string"},
            "sources": {
                "type": "array",
                "items": {"type": "string", "format": "uri"},
                "uniqueItems": True,
            },
        },
        "required": ["status_code", "delivery", "explanation", "sources"],
    },
}

INSTRUCTIONS = """You are a precise promise-auditor. Use web search before answering.
Classify delivery using these codes and labels (must match exactly):
ON_TIME → delivered on time
DELAYED → delayed delivery
NOT_DELIVERED → not delivered
UNMEASURABLE → it is tough to define what a promise delivery is
NO_EVIDENCE → it is easy to define promise delivery but I couldn't find relevant data on it

Rules:
- If promise has a concrete deliverable + deadline: ON_TIME / DELAYED / NOT_DELIVERED.
- If deliverable is concrete but you cannot find reliable evidence: NO_EVIDENCE.
- If the promise is vague/ongoing and not operationalizable: UNMEASURABLE.
Explain briefly (timeline logic + why). Provide sources.
Return one JSON object only, matching the schema.

JSON_SCHEMA = {
    "name": "promise_audit",
    "strict": True,
    "schema": {
        "type": "object",
        "additionalProperties": False,
        "properties": {
            "status_code": {"type": "string", "enum": list(STATUS_MAP.keys())},
            "delivery": {"type": "string", "enum": list(STATUS_MAP.values())},
            "explanation": {"type": "string"},
            "sources": {
                "type": "array",
                "items": {"type": "string", "format": "uri"},
                "uniqueItems": True,
            },
        },
        "required": ["status_code", "delivery", "explanation", "sources"],
    }

STATUS_MAP = {
    "ON_TIME": "delivered on time",
    "DELAYED": "delayed delivery",
    "NOT_DELIVERED": "not delivered",
    "UNMEASURABLE": "it is tough to define what a promise delivery is",
    "NO_EVIDENCE": "it is easy to define promise delivery but I couldn't find relevant data on it",
}
"""

def _row_payload(row: pd.Series) -> str:
    data = {
        "promise_id": row.get("promise_id"),
        "companyname": row.get("companyname"),
        "exec_fullname": row.get("exec_fullname"),
        "mostimportantdateutc": str(row.get("mostimportantdateutc")),
        "promise_verbatim": row.get("1-promise-verbatim"),
        "promise_explain": row.get("2-promise-explain"),
    }
    return (
        "Assess whether the following promise was delivered. Use web search. "
        "Return ONLY the JSON per schema.\n" + json.dumps(data, ensure_ascii=False)
    )


In [10]:
# %%
# --- ASYNC version of the per-row call (minimal change: just 'await' + AsyncOpenAI) ---

async def assess_promise_row_async(row: pd.Series, retries: int = 2, backoff: float = 1.5) -> dict:
    for i in range(retries + 1):
        try:
            r = await aclient.responses.create(
                model="gpt-5",
                instructions=INSTRUCTIONS,
                input=_row_payload(row),
                tools=[{"type": "web_search_preview"}],   # leave as-is (as in your original code)
                reasoning={"effort": "medium"},
                max_output_tokens=20000,
                # Tip: for stricter JSON you could add a response_format with JSON_SCHEMA, but keeping minimal changes
            )
            out = getattr(r, "output_json", None) or json.loads(r.output_text)
            return {
                "promise_id": row.get("promise_id"),
                "status_code": out["status_code"],
                "delivery": out["delivery"],
                "explanation": out["explanation"],
                "sources": out.get("sources", []),
            }
        except Exception as e:
            if i == retries:
                return {
                    "promise_id": row.get("promise_id"),
                    "status_code": "NO_EVIDENCE",
                    "delivery": STATUS_MAP["NO_EVIDENCE"],
                    "explanation": f"API/search error: {e}",
                    "sources": [],
                }
            await asyncio.sleep(backoff * (i + 1))


In [11]:
# %%
# --- Concurrent runner with bounded parallelism + a Jupyter-friendly wrapper ---

async def audit_promises_async(df: pd.DataFrame, concurrency: int = 8) -> pd.DataFrame:
    sem = asyncio.Semaphore(concurrency)
    rows_out = []

    async def _bounded(row):
        async with sem:
            # use a shallow copy so the async task never sees a mutated Series
            return await assess_promise_row_async(row.copy())

    tasks = [asyncio.create_task(_bounded(row)) for _, row in df.iterrows()]

    for fut in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Auditing (async)"):
        result = await fut
        rows_out.append(result)
        print(result)

    return pd.DataFrame(rows_out)


def audit_promises(df: pd.DataFrame, concurrency: int = 8) -> pd.DataFrame:
    """
    Thin wrapper so you can call audit_promises(...) from a normal Jupyter cell
    without writing 'await ...'. Uses nest_asyncio to cooperate with the IPython loop.
    """
    loop = asyncio.get_event_loop()
    # With nest_asyncio applied, run_until_complete works fine in notebooks.
    return loop.run_until_complete(audit_promises_async(df, concurrency))


In [12]:
# %%
# Run a small test sample (same API as before; now concurrent under the hood)
results_df = audit_promises(promises_select.sample(12), concurrency=8)
results_df


Auditing (async):   8%|▊         | 1/12 [00:47<08:37, 47.05s/it]

{'promise_id': '14824_1519832_01', 'status_code': 'UNMEASURABLE', 'delivery': 'it is tough to define what a promise delivery is', 'explanation': 'The 2018-07-27 statement is an open-ended operating policy (to adjust reserve estimates as new data arrives) with no fixed deliverable or deadline, so it cannot be judged on-time/late. Evidence shows the company routinely makes such adjustments in subsequent periods—e.g., the 2023 and 2024 Form 10-Ks report prior-year favorable/unfavorable reserve development, and the Q4 2023 earnings call describes updating reserve estimates each quarter—but this is an ongoing process rather than a discrete promise.', 'sources': ['https://www.sec.gov/Archives/edgar/data/20286/000002028624000014/cinf-20231231.htm', 'https://www.sec.gov/Archives/edgar/data/0000020286/000002028625000009/cinf-20241231.htm', 'https://thetranscript.net/transcript/5726/cincinnati-financial-q4-2023-earnings-call-transcript']}


Auditing (async):  17%|█▋        | 2/12 [01:05<05:00, 30.04s/it]

{'promise_id': '116772_2481403_01', 'status_code': 'ON_TIME', 'delivery': 'delivered on time', 'explanation': 'Evidence from the 2013 filing season shows TaxACT offered exactly what was promised: (1) free preparation, e‑file, and print of federal returns for all taxpayers (no AGI or filing-type restrictions) and (2) a Price Lock Guarantee so early starters paid the price at registration even if they filed later. These terms were published during the 2012–2013 season and reiterated for the 2013 tax year, and the guarantee is also described in corporate filings—indicating the promise was in effect at the time stated.', 'sources': ['https://www.taxact.com/press/2013/taxact-puts-taxpayers-first-in-line-for-efile', 'https://www.taxact.com/press/2013/taxact-2013-easy-fast-free-tax-and-healthcare-planning-for-all-taxpayers', 'https://www.sec.gov/Archives/edgar/data/1068875/000106887516000198/bcor10-k2015.htm', 'https://www.globenewswire.com/news-release/2016/01/19/920079/0/en/TaxAct-Encourage

Auditing (async):  25%|██▌       | 3/12 [01:05<02:28, 16.45s/it]

...

Auditing (async):  92%|█████████▏| 11/12 [02:36<00:16, 16.57s/it]

{'promise_id': '28340_930916_03', 'status_code': 'DELAYED', 'delivery': 'delayed delivery', 'explanation': 'The CEO said on Feb 10, 2016 they would spend about $25M in 2016 to complete major renovations at Riverhead (NY) and Rehoboth Beach (DE). Renovation work at Rehoboth began in April 2016 with an eight‑month Bayside project and local coverage expected completion by Thanksgiving 2016. However, Tanger’s filings show only ~$17.1M companywide was spent on “major outlet center renovations” in 2016, and SEC reports state that construction activities at Riverhead and Rehoboth continued in 2017, with additional spending planned to complete those renovations. This indicates the renovations were not fully completed in 2016 and slipped into 2017, i.e., delivered later than promised.', 'sources': ['https://www.capegazette.com/node/102249', 'https://delawaretoday.com/uncategorized/where-to-shop-this-summer/', 'https://annual-statements.com/company/tanger-inc/annual-report-2017-form-10q-70975', 

Auditing (async): 100%|██████████| 12/12 [02:39<00:00, 13.26s/it]

{'promise_id': '3144_2075241_02', 'status_code': 'NO_EVIDENCE', 'delivery': "it is easy to define promise delivery but I couldn't find relevant data on it", 'explanation': 'The pledge is concrete: at least $500M in incremental spend with Black‑owned suppliers over five years (announced in 2020, implying a 2025 target). However, Coca‑Cola’s public updates disclose total “diverse supplier” spend (e.g., ~$900M in 2022) without breaking out spend specifically with Black‑owned suppliers, and we found no official confirmation that the $500M Black‑owned target has been achieved as of Sep 3, 2025. Therefore, delivery is measurable but not verifiable from public sources.', 'sources': ['https://www.coca-colacompany.com/media-center/coca-cola-accelerates-commitment-to-black-owned-businesses', 'https://nmsdc.org/blog/coca-cola-commits-500-million-in-additional-spending-with-black-owned-suppliers/', 'https://investors.coca-colacompany.com/filings-reports/all-sec-filings/content/0001308179-23-000117

,promise_id,status_code,delivery,explanation,sources
0,14824_1519832_01,UNMEASURABLE,it is tough to define what a promise delivery is,The 2018-07-27 statement is an open-ended oper...,[https://www.sec.gov/Archives/edgar/data/20286...
1,116772_2481403_01,ON_TIME,delivered on time,Evidence from the 2013 filing season shows Tax...,[https://www.taxact.com/press/2013/taxact-puts...
2,8762_1432843_02,UNMEASURABLE,it is tough to define what a promise delivery is,"On April 19, 2018, CEO David S. Taylor said P&...",[https://www.businesswire.com/news/home/201804...
3,162129_101772_02,ON_TIME,delivered on time,Dated milestone: start shaft sinking in calend...,[https://mosaicincanada.com/mosaic-in-canada/N...
4,31673_571026_01,UNMEASURABLE,it is tough to define what a promise delivery is,"The statement (Jan 23, 2014) is a broad, ongoi...",[https://investor.amerisourcebergen.com/news/n...
5,14624_1410711_06,UNMEASURABLE,it is tough to define what a promise delivery is,The 2018 promise commits to “investing” that y...,[https://seekingalpha.com/article/4151737-offi...
6,13071_1460894_02,ON_TIME,delivered on time,"On May 9, 2018, Dr. Berger said RadNet would e...",[https://www.sec.gov/Archives/edgar/data/79052...
7,16597_754037_02,ON_TIME,delivered on time,"Context set a concrete yardstick: on Jan 27, 2...",[https://www.businesswire.com/news/home/201501...
8,10530_613993_01,ON_TIME,delivered on time,Promise made on 2014-04-23 to achieve $85M in ...,[https://www.genomeweb.com/archive/thermo-fish...
9,1440_2665405_04,ON_TIME,delivered on time,"AEP announced on Oct 4, 2022 that it had rebas...","[https://www.aep.com/news/stories/view/8685/, ..."


In [18]:
promises_select.head()

,promise_id,primary_keyword,gvkey,mostimportantdateutc,companyname,exec_fullname,execid,1-promise-verbatim,2-promise-explain,3-promise-horizon-v2,specificity_score
0,10353_45212_05,Shareholder and Stakeholder Trust and Engagement,10353,2010-01-06,"Team, Inc.",Phillip J. Hawk,49854,We have always been and continue to be fully c...,The CEO asserts a continuous commitment to eth...,unclear,1.0
1,11600_45248_01,Risk Management,11600,2010-01-06,"Worthington Industries, Inc.",John P. McConnell,3497,But I assure you we will remain vigilant in se...,The CEO is committing that the company will ma...,unclear,1.0
2,25338_45259_01,Experience Enhancement,25338,2010-01-06,Bed Bath & Beyond Inc.,Steven H. Temares,13517,We always look for ways to enhance our custome...,The CEO is affirming a long‐term commitment to...,unclear,1.0
3,25338_45259_02,Investment,25338,2010-01-06,Bed Bath & Beyond Inc.,Steven H. Temares,13517,While we continue to review and prioritize our...,The CEO is committing to allocate necessary ca...,unclear,1.0
4,146017_45297_01,Pricing and Rate Management,146017,2010-01-06,"Acuity Brands, Inc.",Vernon J. Nagel,11057,"However, as I have said before, we will defend...",The CEO is committing the company to actively ...,unclear,1.0


In [19]:
# randomly select 10% of execids and keep only promises for those execids
execids_sample = promises_select['execid'].drop_duplicates().sample(frac=0.1, random_state=42)
promises_select_10percent = promises_select[promises_select['execid'].isin(execids_sample)]

# %%
promises_select_10percent.head()


,promise_id,primary_keyword,gvkey,mostimportantdateutc,companyname,exec_fullname,execid,1-promise-verbatim,2-promise-explain,3-promise-horizon-v2,specificity_score
20,14172_46250_02,Regulatory and Legal Processes,14172,2010-01-20,Fulton Financial Corporation,"R. Scott Smith, Jr.",35447,We believe that by presenting customers with c...,"Here, the CEO promises that the firm’s compreh...",unclear,1.0
27,140541_46325_01,Expansion Plans,140541,2010-01-20,"Tapestry, Inc.",Lew Frankfort,21393,"Starting in North America, we will open an add...",The CEO clearly commits the company to expandi...,11,5.0
39,16889_46609_01,Risk Management,16889,2010-01-22,Independent Bank Corp.,Christopher Oddleifson,37315,"Looking into 2010, we're going to continue to ...",The CEO is committing that during 2010 the ban...,11,1.0
40,16889_46609_02,Pricing and Rate Management,16889,2010-01-22,Independent Bank Corp.,Christopher Oddleifson,37315,"And of course, we're going to pay very, very c...",The CEO commits to closely monitoring interest...,unclear,1.0
47,11856_46634_01,Conversion Initiatives,11856,2010-01-22,Truist Financial Corporation,Kelly Stuart King,10662,"We've already converted our payrolls, securiti...",In this statement about the Colonial integrati...,3,4.0


In [ ]:
# Run a small test sample (same API as before; now concurrent under the hood)
promises_select_10percent_results = audit_promises(promises_select_10percent, concurrency=25)



In [25]:
# save promises_select_10percent_results to csv
promises_select_10percent_results.to_csv("promises_select_10percent_results.csv", index=False)

# %%
promises_select_10percent_results.head()



,promise_id,status_code,delivery,explanation,sources
0,9203_47104_01,UNMEASURABLE,it is tough to define what a promise delivery is,"Promise made on January 27, 2010 is a broad, o...",[https://www.sec.gov/Archives/edgar/data/10244...
1,25434_46799_01,UNMEASURABLE,it is tough to define what a promise delivery is,"The 2010 pledge is an ongoing ethos (""earn our...",[https://special.seattletimes.com/o/html/coffe...
2,145046_47865_02,UNMEASURABLE,it is tough to define what a promise delivery is,The statement is an open‑ended operating philo...,[https://www.thestreet.com/investing/stocks/we...
3,3863_47450_01,UNMEASURABLE,it is tough to define what a promise delivery is,"On January 28, 2010, the CEO pledged that in 2...",[https://investors.deluxe.com/news-events/pres...
4,9850_47202_01,ON_TIME,delivered on time,"Promise (Jan 27, 2010): complete a 100‑MW biom...",[https://www.power-technology.com/marketdata/p...


In [3]:
promises_select_10percent_results = pd.read_csv("promises_select_10percent_results.csv")

# %%
promises_select_10percent_results.head()

# %%


,promise_id,status_code,delivery,explanation,sources
0,9203_47104_01,UNMEASURABLE,it is tough to define what a promise delivery is,"Promise made on January 27, 2010 is a broad, o...",['https://www.sec.gov/Archives/edgar/data/1024...
1,25434_46799_01,UNMEASURABLE,it is tough to define what a promise delivery is,"The 2010 pledge is an ongoing ethos (""earn our...",['https://special.seattletimes.com/o/html/coff...
2,145046_47865_02,UNMEASURABLE,it is tough to define what a promise delivery is,The statement is an open‑ended operating philo...,['https://www.thestreet.com/investing/stocks/w...
3,3863_47450_01,UNMEASURABLE,it is tough to define what a promise delivery is,"On January 28, 2010, the CEO pledged that in 2...",['https://investors.deluxe.com/news-events/pre...
4,9850_47202_01,ON_TIME,delivered on time,"Promise (Jan 27, 2010): complete a 100‑MW biom...",['https://www.power-technology.com/marketdata/...


In [3]:
# value count for status_code
promises_select_10percent_results['status_code'].value_counts()

# %%


status_code
ON_TIME          3274
UNMEASURABLE     2627
DELAYED           562
NO_EVIDENCE       507
NOT_DELIVERED     354
Name: count, dtype: int64

In [4]:
# merge with promises_select_10percent on promise_id
promises_select_10percent_results_merged = pd.merge(promises_select_10percent_results, promises_select, on=['promise_id'], how='left')

# %%

promises_select_10percent_results_merged.head()


,promise_id,status_code,delivery,explanation,sources,primary_keyword,gvkey,mostimportantdateutc,companyname,exec_fullname,execid,1-promise-verbatim,2-promise-explain,3-promise-horizon-v2,specificity_score
0,9203_47104_01,UNMEASURABLE,it is tough to define what a promise delivery is,"Promise made on January 27, 2010 is a broad, o...",['https://www.sec.gov/Archives/edgar/data/1024...,Efficiency,9203,2010-01-27,"Rockwell Automation, Inc.",Keith D. Nosbusch,17969,We will continue to effectively manage our cos...,"In this statement, the CEO commits to maintain...",unclear,1.0
1,25434_46799_01,UNMEASURABLE,it is tough to define what a promise delivery is,"The 2010 pledge is an ongoing ethos (""earn our...",['https://special.seattletimes.com/o/html/coff...,Customer Relationship Management,25434,2010-01-20,Starbucks Corporation,Howard D. Schultz,887,"Simply stated, we know that we have to earn ou...",The CEO is pledging that Starbucks will contin...,unclear,1.0
2,145046_47865_02,UNMEASURABLE,it is tough to define what a promise delivery is,The statement is an open‑ended operating philo...,['https://www.thestreet.com/investing/stocks/w...,Pricing and Rate Management,145046,2010-01-27,"Elevance Health, Inc.",Angela Rose Fick Braly,32716,We price our business in an actuarially sound ...,This statement commits the company to maintain...,unclear,1.0
3,3863_47450_01,UNMEASURABLE,it is tough to define what a promise delivery is,"On January 28, 2010, the CEO pledged that in 2...",['https://investors.deluxe.com/news-events/pre...,Cost Reduction Initiatives,3863,2010-01-28,Deluxe Corporation,Lee J. Schram,31329,"In 2010, we will not take our eyes off of cost...",The CEO commits that throughout 2010 the compa...,11,1.0
4,9850_47202_01,ON_TIME,delivered on time,"Promise (Jan 27, 2010): complete a 100‑MW biom...",['https://www.power-technology.com/marketdata/...,Clean and Renewable Energy Transition,9850,2010-01-27,The Southern Company,David M. Ratcliffe,19239,We are moving forward with construction of the...,The CEO is committing Southern Power to build ...,35,5.0


In [5]:
# value count for status_code based on different specificity_score
promises_select_10percent_results_merged.groupby(['status_code', 'specificity_score']).size().unstack()

# %%




specificity_score,1.0,2.0,3.0,4.0,5.0
status_code,,,,,
DELAYED,8,37,182,205,130
NOT_DELIVERED,47,56,75,73,103
NO_EVIDENCE,16,63,148,149,131
ON_TIME,276,624,918,819,637
UNMEASURABLE,1934,493,147,31,22


In [6]:
# value counts for status code for specificity score above 3
promises_select_10percent_results_merged[promises_select_10percent_results_merged['specificity_score'] > 2]['status_code'].value_counts()

status_code
ON_TIME          2374
DELAYED           517
NO_EVIDENCE       428
NOT_DELIVERED     251
UNMEASURABLE      200
Name: count, dtype: int64

In [7]:
# just the above but in percentages
promises_select_10percent_results_merged[promises_select_10percent_results_merged['specificity_score'] > 2]['status_code'].value_counts() / len(promises_select_10percent_results_merged[promises_select_10percent_results_merged['specificity_score'] > 2])

status_code
ON_TIME          0.629708
DELAYED          0.137135
NO_EVIDENCE      0.113528
NOT_DELIVERED    0.066578
UNMEASURABLE     0.053050
Name: count, dtype: float64

In [11]:
# show the table above, but percentages for each specificity_score
counts = promises_select_10percent_results_merged.groupby(['status_code', 'specificity_score']).size().unstack(fill_value=0)
col_totals = counts.sum(axis=0)
percentages = counts.div(col_totals, axis=1) * 100

# show total percent size of promises with each specificity score
total_promises = counts.values.sum()
specificity_percents = (col_totals / total_promises) * 100
print("Percent of promises with each specificity score:")
print(specificity_percents)

percentages


Percent of promises with each specificity score:
specificity_score
1.0    31.144184
2.0    17.381212
3.0    20.070999
4.0    17.435827
5.0    13.967777
dtype: float64


specificity_score,1.0,2.0,3.0,4.0,5.0
status_code,,,,,
DELAYED,0.350723,2.906520,12.380952,16.053250,12.707722
NOT_DELIVERED,2.060500,4.399057,5.102041,5.716523,10.068426
NO_EVIDENCE,0.701447,4.948940,10.068027,11.667972,12.805474
ON_TIME,12.099956,49.018068,62.448980,64.134691,62.267840
UNMEASURABLE,84.787374,38.727416,10.000000,2.427565,2.150538
